In [ ]:
import pandas as pd
import os
import re
%pip install openpyxl


### Merge files on Attraction -> output: Each excel for each attraction with cols ('Datetime', 'WaitTime', 'Month', 'Year')
### 0Attraction open
### -1Virtual Queue
### -2Maintenance
###  -3Closed due to weather
### -4Attraction closed
### 91over 90 minutes

In [ ]:


# Folder with the Excel files
folder_path = "/home/ms/hfu/ML/ML_Presentation_Workspace/EU_PARK/europark_raw_files" # Corrected absolute path

# Regex to extract attraction, month, and year from filename
pattern = r'^(.*?) - Queue times in (\w+) (\d{4})\.xlsx'

# Dictionary to hold data for each attraction
attraction_data = {}

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        match = re.match(pattern, filename)
        if match:
            attraction = match.group(1).strip()
            month = match.group(2)
            year = int(match.group(3))
            file_path = os.path.join(folder_path, filename)

            # Read the file
            df = pd.read_excel(file_path)

            # Add metadata
            df['Month'] = month
            df['Year'] = year

            # Rename columns for consistency
            df.columns = ['Datetime', 'WaitTime', 'Month', 'Year']

            # Add to the attraction's list
            if attraction not in attraction_data:
                attraction_data[attraction] = []
            attraction_data[attraction].append(df)



In [ ]:
attraction_data

In [ ]:
# For each attraction, combine all months and save one Excel file
safe_attraction_name_list = []
save_path = '/home/ms/hfu/ML/ML_Presentation_Workspace/EU_PARK/europark_attraction_merged_dfs'
for attraction, dfs in attraction_data.items():
    combined_df = pd.concat(dfs, ignore_index=True)

    # Drop rows where Datetime is missing
    combined_df = combined_df.dropna(subset=['Datetime'])

    # Split Datetime into Date and Time using string operations
    combined_df[['Date', 'Time']] = combined_df['Datetime'].astype(str).str.strip().str.split(' ', expand=True)
    safe_attraction_name = attraction.replace('-', ' ')  # avoid file path issues # Create the output file path
    fil_safe_attraction_name = safe_attraction_name.replace(' ','_')
    safe_attraction_name_list.append(fil_safe_attraction_name)
    # print(safe_attraction_name_list)
    output_file = os.path.join(save_path, f"{fil_safe_attraction_name} - All Queue Times.xlsx")
    combined_df.to_excel(output_file, index=False)
    print(f"✅ Saved merged file for: {attraction} -> {output_file}")

### Merge All attractions in one single excel file

In [ ]:
# Folder where individual attraction Excel files are stored
folder_path = 'EU_PARK/europark_attraction_merged_dfs/'

# Get list of merged attraction files
files = [f for f in os.listdir(folder_path) if f.endswith('All Queue Times.xlsx')]
print(files)

In [ ]:
merged_df = None

In [ ]:
for file in files:
    # Full path to the Excel file
    file_path = os.path.join(folder_path, file)

    # Extract attraction name from filename
    attraction = file.replace(' - All Queue Times.xlsx', '').strip()

    # Read file
    df = pd.read_excel(file_path)
    df = df.drop(columns=['Datetime'], errors='ignore')
    df.rename(columns={'WaitTime': f'{attraction}_WaitTime'}, inplace=True)

    # Merge using pandas only
    if merged_df is None:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on=['Date', 'Time', 'Month', 'Year'], how='outer')

merged_df = merged_df.sort_values(by=['Date', 'Time'])

output_file = os.path.join(folder_path, 'All_Attractions_Queue_Times_By_Date_Time.csv')
merged_df.to_csv(output_file, index=False)

print(f"✅ Final merged file saved to: {output_file}")

In [ ]:
merged_df_dt = pd.read_csv("EU_PARK/europark_attraction_merged_dfs/All_Attractions_Queue_Times_By_Date_Time.csv", index_col=False)

In [ ]:
merged_df_dt.dropna()

In [ ]:
merged_df_dt.info()

In [ ]:
merged_df_dt.isna().sum()

In [ ]:
merged_df_dt = merged_df_dt.dropna(subset=['Time'])

In [ ]:
def report_missing_values(df):
    total_rows = len(df)
    missing_count = df.isna().sum()
    missing_percent = (missing_count / total_rows) * 100

    result = pd.DataFrame({
        'Missing Count': missing_count,
        'Missing %': missing_percent.round(2)
    })
    result = result.sort_values(by='Missing %', ascending=False)

    print(result)  # Only show columns with missing values

report_missing_values(merged_df_dt)

In [ ]:
#merged_df_dt.to_csv("EU_PARK/All_Attractions_Queue_Times_By_Date_Time.csv")

### Merge wind data files to one excel files contating wind data

In [ ]:
def merge_fn_wind_prec_temp(df_path,df_type):
    """
    Parameters:
        df_path (str): Path to folder containing Excel files.
        data_type (str): Type of data ('wind', 'prec', 'temp', etc.)
        
    Returns:
        pd.DataFrame: Combined dataframe with Month and Year columns.
    """
    files = [f for f in os.listdir(df_path) if f.endswith('.xlsx')]
    data_final = []
    for file in files:
        file_path = os.path.join(df_path, file)

        # Extract month and year from filename: 'Wind speed in March 2024.xlsx'
        if df_type == 'wind':
            name_parts = file.replace('.xlsx', '').replace('Wind speed in ', '').strip().split()
        elif df_type == 'prec':
            name_parts = file.replace('.xlsx', '').replace('Precipitation probability in ', '').strip().split()
        elif df_type == 'temp':
            name_parts = file.replace('.xlsx', '').replace('Temperatures in ', '').strip().split()
        else:
            raise ValueError(f"Unknown data_type: {df_type}")
        month = name_parts[0]
        year = name_parts[1]

        # Read the file
        df = pd.read_excel(file_path)

        # Add Month and Year columns
        df['Month'] = month
        df['Year'] = int(year)
        data_final.append(df)

    # Combine all dataframes into one
    wind_df = pd.concat(data_final, ignore_index=True)
    wind_df[['Date', 'Time']] = wind_df['date_time'].astype(str).str.strip().str.split(' ', expand=True)
    wind_df = wind_df.drop(columns=['date_time'])

    return wind_df


In [ ]:
wind_loc = 'EU_PARK/wind_speed/'
final_wind_df = merge_fn_wind_prec_temp(wind_loc,df_type='wind')

In [ ]:
final_wind_df.shape

In [ ]:
final_wind_df.head()

In [ ]:
report_missing_values(final_wind_df)

In [ ]:
import os
print(os.getcwd())

### Merge Precipitation data files to one excel files contating Precipitation data

In [ ]:
precipitation_loc = '/home/ms/hfu/ML/ML_Presentation_Workspace/EU_PARK/Precipitation/'

if not os.path.exists(precipitation_loc):
	raise FileNotFoundError(f"Directory does not exist: {precipitation_loc}")

final_prec_df = merge_fn_wind_prec_temp(precipitation_loc,df_type='prec')


In [ ]:
final_prec_df.shape

In [ ]:
final_prec_df.head()

In [ ]:
report_missing_values(final_prec_df)

### Merge Temperature data files to one excel files contating Temperature data

In [ ]:
temperature_loc = 'EU_PARK/Temperatures/'
final_temp_df = merge_fn_wind_prec_temp(temperature_loc,df_type='temp')


In [ ]:
final_temp_df.shape

In [ ]:
final_temp_df.head()

In [ ]:
report_missing_values(final_temp_df)

### Merge Final Excel of waiting times with wind data, Precipitaiton and Temperature data based on Month ,year,Date Time

In [ ]:
# merged_df_dt : final df for all attraction waiting times
# final_temp_df: final df for all attraction Temperatures
# final_prec_df: final df for all attraction Precipitation
# final_wind_df: final df for all attraction Wind speed in kmh

In [ ]:
merge_keys = ['Month', 'Year', 'Date', 'Time']


In [ ]:
# Step-by-step outer merges using pandas only
merged = pd.merge(merged_df_dt, final_temp_df, on=['Month', 'Year', 'Date', 'Time'], how='outer')
merged = pd.merge(merged, final_prec_df, on=['Month', 'Year', 'Date', 'Time'], how='outer')
merged = pd.merge(merged, final_wind_df, on=['Month', 'Year', 'Date', 'Time'], how='outer')

# Optional: sort for clean structure
final_merged = merged.sort_values(by=['Year', 'Month', 'Date', 'Time']).reset_index(drop=True)



In [ ]:
final_merged.shape

In [ ]:
report_missing_values(final_merged)

In [ ]:
final_merged.columns

In [ ]:
gb = final_merged.groupby(['Month', 'Year', 'ARTHUR_WaitTime']).agg(lambda x: x.isna().sum())

In [ ]:
gb

In [ ]:
final_merged.columns

In [ ]:
final_merged.to_csv("final_df.csv")

In [ ]:
final_merged.groupby

In [ ]:
df = pd.read_csv(output_file)

In [ ]:
report_missing_values(df)

### Add a column for school holiday = 0 or 1 (true or false)

In [ ]:
school_holiday_df = pd.read_csv('/home/ms/hfu/ML/ML_Presentation_Workspace/EU_PARK/holidays/baden_wuerttemberg_school_holidays_2022_2026.csv')

In [ ]:
school_holiday_df.columns

In [ ]:
school_holiday_df.shape

In [ ]:
school_holiday_df.head()

## Merge school holidays into final_merged


In [ ]:
import pandas as pd

# 1. Convert both Date columns to datetime (if not already)
school_holiday_df['Date'] = pd.to_datetime(school_holiday_df['Date'])
final_merged['Date'] = pd.to_datetime(final_merged['Date'], errors='coerce')

# 2. Do a left-merge with indicator
final_merged = pd.merge(
    final_merged,
    school_holiday_df[['Date']],    # only need the Date column
    on='Date',
    how='left',
    indicator=True                  # adds a '_merge' column
)

# 3. Create the flag column (1 if holiday, 0 otherwise)
final_merged['Is_School_Holiday'] = (final_merged['_merge'] == 'both').astype(int)

# 4. Drop the helper '_merge' column
final_merged.drop(columns=['_merge'], inplace=True)

# 5. (Optional) inspect result
print(final_merged[['Date', 'Is_School_Holiday']].drop_duplicates().head())

In [ ]:
final_merged.columns

In [ ]:
final_merged.to_csv("final_df.csv")

### Add column for public holiday =1 or 0

In [ ]:
public_holiday_df = pd.read_csv('/home/ms/hfu/ML/ML_Presentation_Workspace/EU_PARK/holidays/baden_wuerttemberg_public_holidays_2022_2025.csv')

In [ ]:
public_holiday_df.columns

In [ ]:
public_holiday_df.head()


In [ ]:


# 1. Convert both Date columns to datetime (if not already)
public_holiday_df['Date'] = pd.to_datetime(school_holiday_df['Date'])
final_merged['Date'] = pd.to_datetime(final_merged['Date'], errors='coerce')

# 2. Do a left-merge with indicator
final_merged = pd.merge(
    final_merged,
    public_holiday_df[['Date']],    # only need the Date column
    on='Date',
    how='left',
    indicator=True                  # adds a '_merge' column
)

# 3. Create the flag column (1 if holiday, 0 otherwise)
final_merged['Is_Public_Holiday'] = (final_merged['_merge'] == 'both').astype(int)

# 4. Drop the helper '_merge' column
final_merged.drop(columns=['_merge'], inplace=True)

# 5. (Optional) inspect result
print(final_merged[['Date', 'Is_Public_Holiday']].drop_duplicates().head())

In [ ]:
final_merged.columns

In [ ]:
final_merged.to_csv("final_df.csv")

###EDA

In [ ]:
final_merged.info()


In [ ]:
final_merged.head()


In [ ]:
final_merged.tail()


In [ ]:
final_merged.shape

In [ ]:
final_merged.describe()

In [ ]:
final_merged.describe(include='all')

In [ ]:
report_missing_values(final_merged)

### Data visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# filepath: /home/ms/hfu/ML/ML_Presentation_Workspace/merge_df.ipynb
# Add to a new cell
import matplotlib.pyplot as plt
import seaborn as sns

# Example: Histogram for a wait time column (replace with an actual column name)
# Identify wait time columns first
wait_time_columns = [col for col in final_merged.columns if 'WaitTime' in col]
if wait_time_columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(final_merged[wait_time_columns[0]].dropna(), kde=True)
    plt.title(f'Distribution of {wait_time_columns[0]}')
    plt.xlabel('Wait Time')
    plt.ylabel('Frequency')
    plt.show()
else:
    print("No wait time columns found to plot.")


# Example: Histogram for Temperature (if 'Temperature' column exists)
if 'temperature_in_celsius' in final_merged.columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(final_merged['temperature_in_celsius'].dropna(), kde=True)
    plt.title('Distribution of Temperature')
    plt.xlabel('Temperature')
    plt.ylabel('Frequency')
    plt.show()
elif 'temp_value' in final_merged.columns: # Based on your merge_fn_wind_prec_temp
    plt.figure(figsize=(10, 6))
    sns.histplot(final_merged['temp_value'].dropna(), kde=True)
    plt.title('Distribution of Temperature (temp_value)')
    plt.xlabel('Temperature')
    plt.ylabel('Frequency')
    plt.show()
else:
    print("No 'Temperature' or 'temp_value' column found to plot.")



### Exploring Relationships Between Variables
Let's examine how different variables relate to each other, particularly how weather conditions, holidays, and other factors might affect wait times.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Select only numerical columns for correlation analysis
# First, identify the numeric columns in the dataframe
numeric_cols = final_merged.select_dtypes(include=[np.number]).columns.tolist()

# Remove unnecessary numeric columns (like index) if they exist
exclude_cols = []  # Add columns to exclude if needed
numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

# Create a correlation matrix for these numeric columns
if numeric_cols:
    correlation_matrix = final_merged[numeric_cols].corr(method='pearson', numeric_only=True)
    
    # Plot the correlation matrix
    plt.figure(figsize=(14, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title('Correlation Matrix of Numerical Features')
    plt.tight_layout()
    plt.show()
else:
    print("No numerical columns found for correlation analysis")

In [ ]:
# Explore relationship between wait time and holidays
wait_time_columns = [col for col in final_merged.columns if 'WaitTime' in col]
if wait_time_columns and 'Is_School_Holiday' in final_merged.columns:
    # Pick a representative wait time column
    wait_col = wait_time_columns[0]
    
    # Filter out values < 0 (probably maintenance or closed)
    df_filtered = final_merged[final_merged[wait_col] >= 0].copy()
    
    # Calculate average wait time on school holidays vs regular days
    avg_by_holiday = df_filtered.groupby('Is_School_Holiday')[wait_col].agg(['mean', 'median', 'std']).reset_index()
    avg_by_holiday['Is_School_Holiday'] = avg_by_holiday['Is_School_Holiday'].map({0: 'Regular Day', 1: 'School Holiday'})
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Is_School_Holiday', y='mean', data=avg_by_holiday)
    plt.title(f'Average {wait_col} by School Holiday Status')
    plt.ylabel('Average Wait Time (minutes)')
    plt.show()
    
    print(f"Wait time statistics by school holiday status:\n{avg_by_holiday}")

In [ ]:
# Explore wait time by time of day
if wait_time_columns and 'Time' in final_merged.columns:
    # Extract hour from time column
    df_time = final_merged.copy()
    try:
        # Try to extract hour directly if Time is already properly formatted
        df_time['Hour'] = pd.to_datetime(df_time['Time']).dt.hour
    except:
        # If that fails, try a different approach assuming 'Time' is a string like '10:30'
        df_time['Hour'] = df_time['Time'].str.split(':', expand=True)[0].astype(int)
    
    # Filter out negative wait times
    wait_col = wait_time_columns[0]
    df_time = df_time[df_time[wait_col] >= 0]
    
    # Group by hour and calculate average wait time
    hourly_avg = df_time.groupby('Hour')[wait_col].mean().reset_index()
    
    plt.figure(figsize=(12, 6))
    sns.lineplot(x='Hour', y=wait_col, data=hourly_avg, marker='o')
    plt.title(f'Average {wait_col} by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Average Wait Time (minutes)')
    plt.xticks(range(0, 24))
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# Explore the relationship between weather and wait times
if wait_time_columns and 'temp_value' in final_merged.columns:
    wait_col = wait_time_columns[0]
    df_filtered = final_merged[(final_merged[wait_col] >= 0) & (~final_merged['temp_value'].isna())].copy()
    
    plt.figure(figsize=(12, 6))
    sns.scatterplot(x='temp_value', y=wait_col, data=df_filtered, alpha=0.5)
    plt.title(f'Relationship Between Temperature and {wait_col}')
    plt.xlabel('Temperature (°C)')
    plt.ylabel('Wait Time (minutes)')
    
    # Add a trend line
    sns.regplot(x='temp_value', y=wait_col, data=df_filtered, scatter=False, color='red')
    plt.show()
    
    # Calculate correlation
    correlation = df_filtered[['temp_value', wait_col]].corr().iloc[0, 1]
    print(f"Correlation between temperature and {wait_col}: {correlation:.3f}")